# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'01-03-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'01-03-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-01-04 05:22:02,33.93911,67.709953,51526,2191,41727,90.0,Afghanistan,0.806610,4.252222
1,NaN,NaN,NaN,Albania,2021-01-04 05:22:02,41.15330,20.168300,59438,1193,34648,23597.0,Albania,2065.397178,2.007133
2,NaN,NaN,NaN,Algeria,2021-01-04 05:22:02,28.03390,1.659600,100408,2772,67808,29828.0,Algeria,228.975169,2.760736
3,NaN,NaN,NaN,Andorra,2021-01-04 05:22:02,42.50630,1.521800,8192,84,7517,591.0,Andorra,10602.472012,1.025391
4,NaN,NaN,NaN,Angola,2021-01-04 05:22:02,-11.20270,17.873900,17642,408,11223,6011.0,Angola,53.678136,2.312663


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,50810,50886,51039,51280,51350,51405,51526,51526,51526,51526
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,55380,55755,56254,56572,57146,57727,58316,58316,58991,59438
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,97441,97857,98249,98631,98988,99311,99610,99897,100159,100408


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2139,2149,2160,2174,2179,2181,2191,2191,2191,2191
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1134,1143,1153,1164,1170,1174,1181,1181,1190,1193
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2716,2722,2728,2737,2745,2751,2756,2762,2769,2772


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,40444,40784,41096,41441,41543,41612,41727,41727,41727,41727
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,30790,31181,31565,32122,32700,33185,33634,33634,34353,34648
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,65144,65505,65862,66214,66550,66855,67127,67395,67611,67808


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,3990,3999,4029,4065,4105,4164,4190,4239,4268,4305
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,12666,12708,12825,12962,13172,13392,13601,13823,13955,14064
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1396,1398,1406,1417,1462,1492,1514,1517,1528,1530


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,46,46,47,47,47,48,48,50,50,50
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,151,151,152,152,156,160,161,169,169,169
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,32,32,32,32,32,32,32,33,33,33


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
650,1001.0,Autauga,Alabama,US,2021-01-04 05:22:02,32.539527,-86.644082,4305,50,0,4255.0,"Autauga, Alabama, US",7705.525426,1.161440
687,1075.0,Lamar,Alabama,US,2021-01-04 05:22:02,33.779950,-88.096680,1074,15,0,1059.0,"Lamar, Alabama, US",7779.789931,1.396648
688,1077.0,Lauderdale,Alabama,US,2021-01-04 05:22:02,34.901719,-87.656247,6682,70,0,6612.0,"Lauderdale, Alabama, US",7205.944203,1.047591


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,20636663,351580,0
India,10340469,149649,9946867
Brazil,7733746,196018,6904172


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,20636663,351580,0,20285083,2021-01-04 05:22:02,37.936303,-91.379001
India,10340469,149649,9946867,243953,2021-01-04 05:22:02,23.088275,81.806127
Brazil,7733746,196018,6904172,633556,2021-01-04 05:22:02,-12.669522,-48.480493
Russia,3203743,57730,2591937,554076,2021-01-04 05:22:02,54.546312,62.120860
France,2712975,65164,201308,2446503,2021-01-04 05:22:02,6.430808,-34.730285


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,2426930,26638,0
Texas,1810320,28430,0
Florida,1365436,21987,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,2426930,26638,0,2400292,2021-01-04 05:22:02,37.843962,-120.728594
Texas,1810320,28430,0,1781890,2021-01-04 05:22:02,31.660643,-98.653069
Florida,1365436,21987,0,1343449,2021-01-04 05:22:02,28.940755,-82.700744
New York,1023897,38415,0,985482,2021-01-04 05:22:02,42.544151,-75.474183
Illinois,979821,18322,0,961499,2021-01-04 05:22:02,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,818639,10773,0
Illinois,Cook,399609,8436,0
Arizona,Maricopa,342994,5239,0
Florida,Miami-Dade,305734,4251,0
Texas,Harris,245551,3431,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,818639,10773,0,807866,2021-01-04 05:22:02,34.308284,-118.228241,6037.0
Illinois,Cook,399609,8436,0,391173,2021-01-04 05:22:02,41.841448,-87.816588,17031.0
Arizona,Maricopa,342994,5239,0,337755,2021-01-04 05:22:02,33.348359,-112.491815,4013.0
Florida,Miami-Dade,305734,4251,0,301483,2021-01-04 05:22:02,25.611236,-80.551706,12086.0
Texas,Harris,245551,3431,0,242120,2021-01-04 05:22:02,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-01,51526,58316,99897,8117,17568,159,1629594,159738,28460,362911,...,2549689,19753,77182,1,113884,1474,139223,2101,20997,14084
2021-01-02,51526,58991,100159,8166,17608,159,1634834,159798,28484,364302,...,2607542,20275,77182,1,114083,1482,140287,2101,21230,14491
2021-01-03,51526,59438,100408,8192,17642,160,1640718,160027,28504,365768,...,2662699,20823,77258,1,114230,1494,141219,2101,21582,15265


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-01,2191,1181,2762,84,405,5,43319,2828,909,6261,...,74237,193,614,0,1030,35,1418,610,390,369
2021-01-02,2191,1190,2769,84,407,5,43375,2836,909,6275,...,74682,201,614,0,1032,35,1446,610,392,377
2021-01-03,2191,1193,2772,84,408,5,43482,2850,909,6324,...,75137,204,614,0,1034,35,1470,610,394,380


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-01,41727,33634,67395,7463,11146,148,1426676,143355,25770,334901,...,5679,13990,75244,1,107901,1325,118926,1396,18773,11347
2021-01-02,41727,34353,67611,7463,11189,148,1447092,143640,25778,336922,...,5744,14592,75244,1,108086,1337,120403,1396,18942,11363
2021-01-03,41727,34648,67808,7517,11223,148,1452960,144091,25778,338831,...,6094,15125,75437,1,108268,1339,121563,1399,19083,11574


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,3990,3999,4029,4065,4105,4164,4190,4239,4268,4305
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1007,1009,1012,1017,1025,1037,1056,1065,1069,1074
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,6048,6039,6117,6162,6230,6371,6478,6566,6613,6682
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1921,1924,1939,1958,1978,2005,2044,2082,2102,2115
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,9779,9800,9856,9928,10047,10159,10320,10450,10508,10560


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-01,365747,46986,0,530267,229442,2336201,337161,185708,58064,49,...,586802,1779595,276612,7412,2036,354766,246752,87820,522523,44409
2021-01-02,369458,47801,0,539150,231442,2388557,339172,190120,58873,49,...,604132,1793453,281654,7689,2041,358755,246752,89327,523652,44573
2021-01-03,371934,48098,0,556384,233475,2426930,341250,190120,59484,49,...,608297,1810320,283473,7793,2042,363765,255396,91058,526245,44875


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-01-01        4239   13823    1517  1854   4693     888   1522    9584   
2021-01-02        4268   13955    1528  1863   4729     892   1530    9692   
2021-01-03        4305   14064    1530  1882   4746     900   1546    9731   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-01         2366     1429  ...    2021    440     2543      597   
2021-01-02         2386     1435  ...    2035    441     2555      598   
2021-01-03         2402     1438  ...    2081    449     2573      599   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-01           2966  2138  1558          0      781    476  
2021-01-02           2992  2145  1569          0      782    477  
2021-01-03           3020  2159  1570          0      783    477  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-01,4872,206,0,9015,3711,26286,4873,5995,930,0,...,6907,28253,1269,136,23,5081,3461,1361,5254,438
2021-01-02,4872,215,0,9061,3729,26542,4912,6099,930,0,...,6970,28338,1294,139,23,5117,3461,1373,5256,438
2021-01-03,4873,215,0,9061,3749,26638,4934,6099,930,0,...,7025,28430,1301,140,23,5124,3459,1376,5261,438


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-01-01          50     169      33   46     63      22     45     157   
2021-01-02          50     169      33   46     63      22     45     157   
2021-01-03          50     169      33   46     63      22     45     157   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-01-01           63       22  ...      10      8       17        6   
2021-01-02           63       22  ...      10      8       17        6   
2021-01-03           63       22  ...      10      8       17        6   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-01             16     4     7          0       19      2  
2021-01-02             16     4     7          0       19      2  
2021-01-03             16     4     7          0       19      2  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-01,0.0,0.000000,0.002881,0.008448,0.000855,0.000000,0.002510,0.002064,0.001231,0.005809,...,0.021415,0.033161,0.001583,0.0,0.002871,0.006143,0.008833,0.000953,0.013124,0.015649
2021-01-02,0.0,0.011575,0.002623,0.006037,0.002277,0.000000,0.003216,0.000376,0.000843,0.003833,...,0.022690,0.026426,0.000000,0.0,0.001747,0.005427,0.007642,0.000000,0.011097,0.028898
2021-01-03,0.0,0.007577,0.002486,0.003184,0.001931,0.006289,0.003599,0.001433,0.000702,0.004024,...,0.021153,0.027028,0.000985,0.0,0.001289,0.008097,0.006644,0.000000,0.016580,0.053412


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-01,0.0,0.000000,0.002177,0.0,0.000000,0.0,0.001711,0.001771,0.0,0.006268,...,0.008353,0.066298,0.0,NaN,0.001946,0.0,0.012857,0.0,0.005155,0.016529
2021-01-02,0.0,0.007621,0.002534,0.0,0.004938,0.0,0.001293,0.002829,0.0,0.002236,...,0.005994,0.041451,0.0,NaN,0.001942,0.0,0.019746,0.0,0.005128,0.021680
2021-01-03,0.0,0.002521,0.001083,0.0,0.002457,0.0,0.002467,0.004937,0.0,0.007809,...,0.006092,0.014925,0.0,NaN,0.001938,0.0,0.016598,0.0,0.005102,0.007958


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-01,0.0,0.000000,0.003992,0.004171,0.009236,0.0,0.000000,0.003880,0.000311,0.005854,...,0.036503,0.038759,0.004016,0.0,0.002956,0.000000,0.014874,0.001435,0.006056,0.008622
2021-01-02,0.0,0.021377,0.003205,0.000000,0.003858,0.0,0.014310,0.001988,0.000310,0.006035,...,0.011446,0.043031,0.000000,0.0,0.001715,0.009057,0.012419,0.000000,0.009002,0.001410
2021-01-03,0.0,0.008587,0.002914,0.007236,0.003039,0.0,0.004055,0.003140,0.000000,0.005666,...,0.060933,0.036527,0.002565,0.0,0.001684,0.001496,0.009634,0.002149,0.007444,0.018569


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-01,0.012516,0.000000,NaN,0.019338,0.019117,0.019727,0.009171,0.000000,0.010582,0.0,...,0.000000,0.007523,0.000000,0.000000,0.002462,0.014823,0.000000,0.029133,0.004006,0.000000
2021-01-02,0.010146,0.017346,NaN,0.016752,0.008717,0.022411,0.005965,0.023758,0.013933,0.0,...,0.029533,0.007787,0.018228,0.037372,0.002456,0.011244,0.000000,0.017160,0.002161,0.003693
2021-01-03,0.006702,0.006213,NaN,0.031965,0.008784,0.016065,0.006127,0.000000,0.010378,0.0,...,0.006894,0.009405,0.006458,0.013526,0.000490,0.013965,0.035031,0.019378,0.004952,0.006775


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-01      0.011695  0.016322  0.001982  0.010905  0.011204  0.033760   
2021-01-02      0.006841  0.009549  0.007251  0.004854  0.007671  0.004505   
2021-01-03      0.008669  0.007811  0.001309  0.010199  0.003595  0.008969   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-01      0.009284  0.009480  0.010679  0.010608  ...  0.000000   
2021-01-02      0.005256  0.011269  0.008453  0.004199  ...  0.006927   
2021-01-03      0.010458  0.004024  0.006706  0.002091  ...  0.022604   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-01      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2021-01-02      0.002273  0.004719  0.001675   0.008766  0.003274  0.007060   
2021-01-03      0.018141  0.007045  0.001672   0.009358  0.006527  0.000637   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-01            NaN  0.000000  0.000000  
2021-01-02            NaN  0.001280  0.002101  
2021-01-03            NaN  0.001279  0.000000  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-01,0.009323,0.000000,NaN,0.017035,0.009521,0.015021,0.012256,0.000000,0.00432,NaN,...,0.000000,0.011058,0.000000,0.000000,0.0,0.009738,0.000000,0.017190,0.002289,0.0
2021-01-02,0.000000,0.043689,NaN,0.005103,0.004850,0.009739,0.008003,0.017348,0.00000,NaN,...,0.009121,0.003009,0.019701,0.022059,0.0,0.007085,0.000000,0.008817,0.000381,0.0
2021-01-03,0.000205,0.000000,NaN,0.000000,0.005363,0.003617,0.004479,0.000000,0.00000,NaN,...,0.007891,0.003247,0.005410,0.007194,0.0,0.001368,-0.000578,0.002185,0.000951,0.0


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                \
Admin2           Autauga   Baldwin  Barbour Bibb Blount Bullock Butler   
2021-01-01      0.041667  0.049689  0.03125  0.0    0.0     0.0    0.0   
2021-01-02      0.000000  0.000000  0.00000  0.0    0.0     0.0    0.0   
2021-01-03      0.000000  0.000000  0.00000  0.0    0.0     0.0    0.0   

Province_State                             ... Wyoming                  \
Admin2          Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2021-01-01      0.00641      0.0      0.0  ...     0.0    0.0      0.0   
2021-01-02      0.00000      0.0      0.0  ...     0.0    0.0      0.0   
2021-01-03      0.00000      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-01-01          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-01-02          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-01-03          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-01,0.001043,0.004900,0.003081,0.007992,0.003112,0.00222,0.004672,0.002598,0.001006,0.006451,...,0.021540,0.033237,0.001296,0.0,0.003000,0.005189,0.009051,0.000824,0.012712,0.016345
2021-01-02,0.000521,0.008237,0.002852,0.007014,0.002694,0.00111,0.003944,0.001487,0.000925,0.005142,...,0.022115,0.029832,0.000648,0.0,0.002373,0.005308,0.008347,0.000412,0.011905,0.022622
2021-01-03,0.000261,0.007907,0.002669,0.005099,0.002313,0.00370,0.003772,0.001460,0.000813,0.004583,...,0.021634,0.028430,0.000816,0.0,0.001831,0.006703,0.007495,0.000206,0.014242,0.038017


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-01,0.001773,0.002798,0.002183,0.000785,0.001029,2.384186e-07,0.002216,0.003535,0.000017,0.009853,...,0.009936,0.052997,0.000229,NaN,0.002573,1.495992e-38,0.015606,0.000335,0.004518,0.012490
2021-01-02,0.000886,0.005209,0.002359,0.000392,0.002984,1.192093e-07,0.001754,0.003182,0.000009,0.006044,...,0.007965,0.047224,0.000115,NaN,0.002257,7.479959e-39,0.017676,0.000167,0.004823,0.017085
2021-01-03,0.000443,0.003865,0.001721,0.000196,0.002720,5.960464e-08,0.002110,0.004059,0.000004,0.006927,...,0.007029,0.031075,0.000057,NaN,0.002098,3.739979e-39,0.017137,0.000084,0.004962,0.012521


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-01-01,0.001549,0.007337,0.004244,0.004782,0.013973,0.001083,0.001105,0.005280,0.000235,0.006202,...,0.027336,0.039207,0.003072,0.0,0.002959,0.001597,0.016235,0.001663,0.006311,0.010122
2021-01-02,0.000775,0.014357,0.003725,0.002391,0.008916,0.000542,0.007708,0.003634,0.000273,0.006118,...,0.019391,0.041119,0.001536,0.0,0.002337,0.005327,0.014327,0.000831,0.007657,0.005766
2021-01-03,0.000387,0.011472,0.003319,0.004813,0.005977,0.000271,0.005881,0.003387,0.000136,0.005892,...,0.040162,0.038823,0.002050,0.0,0.002010,0.003411,0.011981,0.001490,0.007550,0.012167


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-01,0.012249,0.004298,NaN,0.015965,0.015626,0.017162,0.008627,0.005789,0.011415,8.556445e-89,...,0.005448,0.009058,0.006544,0.007489,0.005855,0.014044,0.006324,0.021742,0.005395,0.002960
2021-01-02,0.011198,0.010822,NaN,0.016358,0.012172,0.019786,0.007296,0.014773,0.012674,4.278223e-89,...,0.017491,0.008423,0.012386,0.022431,0.004155,0.012644,0.003162,0.019451,0.003778,0.003327
2021-01-03,0.008950,0.008518,NaN,0.024162,0.010478,0.017926,0.006711,0.007387,0.011526,2.139111e-89,...,0.012192,0.008914,0.009422,0.017978,0.002323,0.013305,0.019096,0.019415,0.004365,0.005051


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2021-01-01      0.010334  0.015771  0.009624  0.011105  0.011062  0.023515   
2021-01-02      0.008588  0.012660  0.008438  0.007980  0.009367  0.014010   
2021-01-03      0.008628  0.010235  0.004873  0.009089  0.006481  0.011489   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-01      0.009381  0.008354  0.012253  0.009314  ...  0.007303   
2021-01-02      0.007319  0.009811  0.010353  0.006756  ...  0.007115   
2021-01-03      0.008888  0.006918  0.008529  0.004423  ...  0.014860   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-01-01      0.006823  0.003758  0.003233   0.003598  0.003242  0.005422   
2021-01-02      0.004548  0.004238  0.002454   0.006182  0.003258  0.006241   
2021-01-03      0.011344  0.005642  0.002063   0.007770  0.004893  0.003439   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-01-01           -1.0  0.005844  0.002398  
2021-01-02           -1.0  0.003562  0.002249  
2021-01-03           -1.0  0.002420  0.001125  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-01-01,0.008993,0.005352,NaN,0.015370,0.010553,0.016800,0.012239,0.002996,0.006877,NaN,...,0.007177,0.010851,0.005781,0.009497,1.201541e-19,0.009839,0.008429,0.017169,0.005888,0.023083
2021-01-02,0.004496,0.024521,NaN,0.010236,0.007702,0.013270,0.010121,0.010172,0.003439,NaN,...,0.008149,0.006930,0.012741,0.015778,6.007704e-20,0.008462,0.004214,0.012993,0.003134,0.011542
2021-01-03,0.002351,0.012260,NaN,0.005118,0.006533,0.008443,0.007300,0.005086,0.001719,NaN,...,0.008020,0.005088,0.009075,0.011486,3.003852e-20,0.004915,0.001818,0.007589,0.002043,0.005771


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                   \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount  Bullock   
2021-01-01      0.023878  0.031381  0.015629  0.011905  0.010868  0.02500   
2021-01-02      0.011939  0.015691  0.007815  0.005953  0.005434  0.01250   
2021-01-03      0.005969  0.007845  0.003907  0.002976  0.002717  0.00625   

Province_State                                              ...   Wyoming  \
Admin2                Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-01-01      7.049360e-07  0.009952  0.006092 -0.005436  ...  0.107143   
2021-01-02      3.524680e-07  0.004976  0.003046 -0.002718  ...  0.053571   
2021-01-03      1.762340e-07  0.002488  0.001523 -0.001359  ...  0.026786   

Province_State                                                        \
Admin2                Platte  Sheridan Sublette Sweetwater     Teton   
2021-01-01      3.405019e-11  0.017711   0.0500   0.016667  0.098958   
2021-01-02      1.702510e-11  0.008855   0.0250   0.008333  0.049479   
2021-01-03      8.512549e-12  0.004428   0.0125   0.004167  0.024740   

Province_State                                                   
Admin2                 Uinta Unassigned  Washakie        Weston  
2021-01-01      1.314717e-08  -0.501961  0.057336  2.328306e-10  
2021-01-02      6.573585e-09  -0.501961  0.028668  1.164153e-10  
2021-01-03      3.286793e-09  -0.501961  0.014334  5.820765e-11  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210103,AK,46552,NaN,1247285,NaN,totalTestsViral,1293837,87.0,1033.0,...,1293837,0,3,258863ce822db7c57f566042b9752f6de5ced1c7,0,0,0,0,0,NaN
1,20210103,AL,371934,72134.0,1600270,NaN,totalTestsPeopleViral,1900070,2885.0,34373.0,...,1972204,1,0,5e4846bf713a33a2d623f121327339de5fc6ba63,0,0,0,0,0,NaN
2,20210103,AR,231442,40276.0,1888622,NaN,totalTestsViral,2079788,1216.0,11487.0,...,2120064,0,0,55cb7223f2cec4b34f1bbd366fca253b7f2e0dc3,0,0,0,0,0,NaN
3,20210103,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,fe1ba1cd64ec5441ec2ad5eff24d48e3e563e4b0,0,0,0,0,0,NaN
4,20210103,AZ,556384,28025.0,2394408,NaN,totalTestsViral,5257472,4557.0,39728.0,...,2950792,0,289,62bfa415a0eb3c58156b40ec39d9a737d0e2979d,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

/Users/Brian/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210103,AK,46552.0,NaN,1247285.0,NaN,totalTestsViral,1293837.0,87.0,1033.0,...,1293837,0,3,258863ce822db7c57f566042b9752f6de5ced1c7,0,0,0,0,0,NaN
1,20210103,AL,371934.0,72134.0,1600270.0,NaN,totalTestsPeopleViral,1900070.0,2885.0,34373.0,...,1972204,1,0,5e4846bf713a33a2d623f121327339de5fc6ba63,0,0,0,0,0,NaN
2,20210103,AR,231442.0,40276.0,1888622.0,NaN,totalTestsViral,2079788.0,1216.0,11487.0,...,2120064,0,0,55cb7223f2cec4b34f1bbd366fca253b7f2e0dc3,0,0,0,0,0,NaN
3,20210103,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,fe1ba1cd64ec5441ec2ad5eff24d48e3e563e4b0,0,0,0,0,0,NaN
4,20210103,AZ,556384.0,28025.0,2394408.0,NaN,totalTestsViral,5257472.0,4557.0,39728.0,...,2950792,0,289,62bfa415a0eb3c58156b40ec39d9a737d0e2979d,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-03,AK,46552,NaN,1247285,NaN,totalTestsViral,1293837,87.0,1033.0,NaN,...,1293837,0,3,258863ce822db7c57f566042b9752f6de5ced1c7,0,0,0,0,0,NaN
2021-01-03,AL,371934,72134.0,1600270,NaN,totalTestsPeopleViral,1900070,2885.0,34373.0,NaN,...,1972204,1,0,5e4846bf713a33a2d623f121327339de5fc6ba63,0,0,0,0,0,NaN
2021-01-03,AR,231442,40276.0,1888622,NaN,totalTestsViral,2079788,1216.0,11487.0,393.0,...,2120064,0,0,55cb7223f2cec4b34f1bbd366fca253b7f2e0dc3,0,0,0,0,0,NaN
2021-01-03,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,fe1ba1cd64ec5441ec2ad5eff24d48e3e563e4b0,0,0,0,0,0,NaN
2021-01-03,AZ,556384,28025.0,2394408,NaN,totalTestsViral,5257472,4557.0,39728.0,1081.0,...,2950792,0,289,62bfa415a0eb3c58156b40ec39d9a737d0e2979d,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-01-03,AK,46552.0,NaN,1247285.0,NaN,totalTestsViral,1293837.0,87.0,1033.0,NaN,...,1293837,0,3,258863ce822db7c57f566042b9752f6de5ced1c7,0,0,0,0,0,NaN
2021-01-03,AL,371934.0,72134.0,1600270.0,NaN,totalTestsPeopleViral,1900070.0,2885.0,34373.0,NaN,...,1972204,1,0,5e4846bf713a33a2d623f121327339de5fc6ba63,0,0,0,0,0,NaN
2021-01-03,AR,231442.0,40276.0,1888622.0,NaN,totalTestsViral,2079788.0,1216.0,11487.0,393.0,...,2120064,0,0,55cb7223f2cec4b34f1bbd366fca253b7f2e0dc3,0,0,0,0,0,NaN
2021-01-03,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,fe1ba1cd64ec5441ec2ad5eff24d48e3e563e4b0,0,0,0,0,0,NaN
2021-01-03,AZ,556384.0,28025.0,2394408.0,NaN,totalTestsViral,5257472.0,4557.0,39728.0,1081.0,...,2950792,0,289,62bfa415a0eb3c58156b40ec39d9a737d0e2979d,0,0,0,0,0,NaN
